### Постановка задачи

На основе существующих открытых обученных моделей (```CodeBERT```, ```InCoder```) собрать решение для суммаризации кода. Поскольку ```InCoder``` из коробки может генерировать описание для кода, то можно не использовать ```CodeBERT``` на предварительном этапе для получения эмбеддингов.

Также стоит отметить, что ```InCoder``` умеет работать не только с питоном, но и со многими другими языками. В прошлой задаче, где нужно было сгенерировать код, использовался питон. Здесь, по сути в обратной задаче, давайте использовать другой язык.

Воспроизвести результат можно не запуская код, для этого есть [демка](https://huggingface.co/spaces/facebook/incoder-demo)

In [1]:
import json, pathlib
import pandas as pd
import numpy as np

Сниппеты кода возьму из задачи на представление кода, потому что генератор этих сниппетов был написан вручную и я хорошо представляю какую задачу решает какждый кусок кода. Код написан на ```Java```. Плюс сами сниппеты зашумлены, это усложнит задачу. Также не придется тратить дополнительное время на понимание задачи из стороннего датасета, там не всегда указано какая задача решается. 

### Исследование

In [2]:
DATASET_PATH = pathlib.Path("dataset/snippets.json")

with open(DATASET_PATH, "r") as data:
    samples = json.load(data)

In [4]:
factorial_snippets = samples["factorial_snippets"]
binsearch_snippets = samples["binsearch_snippets"]
sort_snippets = samples["sort_snippets"]

Загружаем ```InCoder``` и метод с помощью которого можно будет получить описание того что происходит в коде

In [6]:
from incoder import infill

loading model
loading tokenizer
loading complete


In [59]:
def code_to_docstring(code_snippet: str, max_to_generate=128, temperature=0.2):
    parts = code_snippet.split("<insert>")
    result = infill(parts, max_to_generate=max_to_generate, temperature=temperature)
    return result["text"]

Протестируем работоспособность

In [60]:
example = '''\
def count_words(filename):
    """ <insert> """
    counts = Counter()
    with open(filename) as file:
        for line in file:
            words = line.split(' ')
            counts.update(words)
    return counts'''

In [61]:
docstring = code_to_docstring(example)
print(docstring)

def count_words(filename):
    """ Count words in a file """
    counts = Counter()
    with open(filename) as file:
        for line in file:
            words = line.split(' ')
            counts.update(words)
    return counts


def count_sentences(filename):
    """ Count sentences in a file  """
    counts = Counter()
    with open(filename) as file:
        for line in file:
            words = line.split(' ')
            counts.update(words)
    return counts


Модель действительно выдает осмысленные варианты, иногда неправда конечно, но направление правильное. В этой задаче гипотеза примерно такая же, как и в генерации кода: кажется что модель ```InCoder``` хорошо справится с классическими задачами.

### Задача факториал

In [62]:
%%time
from ipywidgets import IntProgress
from IPython.display import display

N = len(factorial_snippets)
docstrings = []

progress = IntProgress(min=0, max=N)
display(progress)

for i in range(N):
    snippet = factorial_snippets[str(i)]
    snippet_lines = snippet.split("\n")
    snippet_lines.insert(0, '/**\n<insert>\n*/')
    snippet = "\n".join(snippet_lines)
    
    docstring = code_to_docstring(snippet)
    docstrings.append(docstring)
    progress.value += 1

IntProgress(value=0, max=25)

CPU times: user 21min 1s, sys: 3.12 s, total: 21min 4s
Wall time: 10min 32s


In [99]:
import re

success = dict()

for i, ds in enumerate(docstrings):
    mo = re.search(".+factorial.+", ds)
    if mo:
        success[i] = mo.group()

In [100]:
success

{2: ' * Returns the factorial of a number.',
 5: ' * factorial function',
 10: '* Function to calculate factorial of a number.',
 12: ' * Function to calculate the factorial of a number',
 13: '* Function to return factorial of a number',
 16: ' * Recursive function to calculate the factorial of a number',
 18: ' * Function to calculate factorial of a given number'}

Среди $25$ сниппетов с факториалом модель смогла верно предсказать для $7$. Все успешные варианты правильные.

### Задача сортировка

In [101]:
%%time

N = len(sort_snippets)
docstrings = []

progress = IntProgress(min=0, max=N)
display(progress)

for i in range(N):
    snippet = sort_snippets[str(i)]
    snippet_lines = snippet.split("\n")
    snippet_lines.insert(0, '/**\n<insert>\n*/')
    snippet = "\n".join(snippet_lines)
    
    docstring = code_to_docstring(snippet)
    docstrings.append(docstring)
    progress.value += 1

IntProgress(value=0, max=25)

CPU times: user 23min 2s, sys: 2.77 s, total: 23min 5s
Wall time: 11min 32s


In [110]:
import re

success_sort = dict()

for i, ds in enumerate(docstrings):
    mo = re.search(".+sort.+", ds)
    if mo:
        success_sort[i] = mo.group()

In [111]:
success_sort

{0: ' *            The array of numbers to sort.',
 5: 'long sort(int arr[], int l, int r) {',
 7: '* prints the sorted array',
 8: '    sort(new long[] { 1, 3, 6, 5, 7, 9, 2, 8, 4, 9, 7, 1, 5, 6, 4, 7, 8 });',
 9: '    sort(nums, 0, nums.length - 1);',
 12: '    sort(new int[] {1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97});',
 13: '    sort(new int[] { 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, ',
 14: 'long sort(long n[]) {',
 15: 'int sort(int a[]) {',
 16: '    sort(new int[] {4, 3, 2, 1});',
 22: '<|/ q tags=java,arrays,sorting |>',
 23: '    sort(new int[] {1, 3, 5, 7, 9});',
 24: '<|/ a tags=sorting,java |>'

Есть правильные ответы, но очень много бреда. Поэтому давайте здесь копнем глубже. Можно посмотреть на такое

In [122]:
print(docstrings[2])

/**
 * Function to find out whether a number is prime or not
 * 
 * Time complexity: O(n)
 * Space complexity: O(1)
 * 
 * Input: 
 * n: array of numbers to be checked
 * l: lower bound of the search
 * r: upper bound of the search
 * 
 * Output: 
 * true if number is prime, false otherwise
*/
long f(long n[], long l, long r) {
    if (l < r) {
        int m = l + (r - l) / 2;
        
        f(n, l, m);
        f(n, m + 1, r);

        int n1 = m - l + 1;
        int n2 = r - m;
        int nums_[] = new int[4];
        int L[] = new int[n1];
        int R[] = new int[n2];

        for (int i = 0; i < n1; ++i)
            L[i] = n[l + i];
        for (int j = 0; j < n2; ++j)
            R[j] = n[m + 1 + j];
        long item_ = 4;
        int i = 0, j = 0;

        int k = l;
        while (i < n1 && j < n2) {
            if (L[i] <= R[j]) {
                n[k] = L[i];
                int nums_[] = new int[4];
                i++;
            }
            else {
                n[k

Вообще мимо. На самом деле там сортировка слиянием. Или такое

In [137]:
print(docstrings[20])

/**
 * Factorial
*/
long factorial(long n[]) {
    int n = n.length;
    
    for (int i = 1; i < n; ++i) {
        long key = n[i];
        boolean b1_ = Boolean.parseBoolean("true");
        int j = i - 1;
        boolean b1_ = Boolean.parseBoolean("true");
        while (j >= 0 && n[j] > key) {
            n[j + 1] = n[j];
            long item_ = 4;
            j = j - 1;
        }
        
        n[j + 1] = key;
    }
}


Здесь функция называется факториал, но на самом деле это сортировка вставками. И модель на это повелась. То есть можно предположить что **модель не очень устойчива к шуму** и часто может не вникать в семантику. Еще есть такое

In [143]:
print(docstrings[1])

/**
 * 
 * 
 */
public static void main(String[] args) {
    
}

/**
 * 
 * 
 */
public static void main(String[] args) {
    
}

/**
 * 
 * 
 */
public static void main(String[] args) {
    
}

/**
 * 
 * 
 */
public static void main(String[] args) {
    
}

/**
 * 
 * 
 */
public static void main(String[] args) {
    
}

/**
 * 
 * 
 */
public static void main(String[] args) {
    
}

/**
 * 
 * 
 */
public static void main(String[] args) {
    
}

/**
 * 

*/
long s(int number[], int l, int r) {
    if (l < r) {
        int m = l + (r - l) / 2;
        
        s(number, l, m);
        s(number, m + 1, r);

        int n1 = m - l + 1;
        int n2 = r - m;
        boolean b1_ = Boolean.parseBoolean("true");
        int L[] = new int[n1];
        int R[] = new int[n2];

        for (int i = 0; i < n1; ++i)
            L[i] = number[l + i];
        for (int j = 0; j < n2; ++j)
            R[j] = number[m + 1 + j];
        string greet_ = "hello world";
        int i = 0, j = 0;

   

А это вообще полнейший бред. 

То есть **модель просто выдает шаблоны, паттерны**, не пытается вникать в суть. Подробнее про это написал в последней ячейке. Бинпоиск даже нет интереса смотреть, потому что повалилось все здесь.

Теперь давайте возьмем что-то более сложное, какую-нибудь олимпиадную задачу с понятной идеей. Здесь гипотеза такая, что ```InCoder``` скорее не сможет справиться, чем сгенерирует что-то осмысленное. Задача на [перестановки](https://leetcode.com/problems/permutations/description/)

In [112]:
leetcode = '''\
/**
<insert>
*/
public List<List<Integer>> permute(int[] nums) {
   List<List<Integer>> list = new ArrayList<>();
   // Arrays.sort(nums); // not necessary
   backtrack(list, new ArrayList<>(), nums);
   return list;
}

private void backtrack(List<List<Integer>> list, List<Integer> tempList, int [] nums){
   if(tempList.size() == nums.length){
      list.add(new ArrayList<>(tempList));
   } else{
      for(int i = 0; i < nums.length; i++){ 
         if(tempList.contains(nums[i])) continue; // element already exists, skip
         tempList.add(nums[i]);
         backtrack(list, tempList, nums);
         tempList.remove(tempList.size() - 1);
      }
   }
}'''

In [113]:
docstring = code_to_docstring(leetcode)
print(docstring)

/**
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 
 * 

*/
public List<List<Integer>> permute(int[] nums) {
   List<List<Integer>> list = new ArrayList<>();
   // Arrays.sort(nums); // not necessary
   backtrack(list, new ArrayList<>(), nums);
   return list;
}

private void backtrack(List<List<Integer>> list, List<Integer> tempList, int [] nums){
   if(tempList.size() == nums.length){
      list.add(new ArrayList<>(tempList));
   } else{
      for(int i = 0; i < nums.length; i++){ 
         if(tempList.contains(nums[i])) continue; // element already exists, skip
         tempList.add(nums[i]);
         backtrack(list, tempList, nums);
         tempList.remove(tempList.size() - 1);
      }
   }
}


Интересно :) Еще раз

In [114]:
docstring = code_to_docstring(leetcode)
print(docstring)

/**
* Given an array of integers, return all possible permutations.
*
* Example:
*
*     Input: nums = [1,2,3]
*     Output: [[1],[2],[3]],
*            [[1],[3],[2]],
*            [[2],[1],[3]],
*            [[2],[3],[1]],
*            [[3],[1],[2]],
*            [[3],[2],[1]]
*
* Note:
*
* The number of permutations is <=10^5
*
* Follow up:
*
* Could you solve it in O(n) time and O(n) space?
*
* Follow up:
*
* Could you solve it in O(n) 
*/
public List<List<Integer>> permute(int[] nums) {
   List<List<Integer>> list = new ArrayList<>();
   // Arrays.sort(nums); // not necessary
   backtrack(list, new ArrayList<>(), nums);
   return list;
}

private void backtrack(List<List<Integer>> list, List<Integer> tempList, int [] nums){
   if(tempList.size() == nums.length){
      list.add(new ArrayList<>(tempList));
   } else{
      for(int i = 0; i < nums.length; i++){ 
         if(tempList.contains(nums[i])) continue; // element already exists, skip
         tempList.add(nums[i]);
         b

А вот это уже намного лучше, НО здесь не проявляется самостоятельность модели, потому что видно, что описание взято напрямую из платформы ```Leetcode```. Это говорит о том, что **```InCoder``` сильно опирается на ранее полученные данные**. Такая же ситуция была с генерацией кода - мало самоятоятельности и как следствие нельзя написать что-то существенное новое. То есть **отсутсвует способность к индуктивности**. Тут скорее стерильная дедукция, когда собирается новый кирпичик на основании старых, новые знания не получаются.

Чтобы это проверить можно взять новую задачу с ```Leetcode```, пусть даже не сильно сложную, и модель не сможет определить что происходит в коде, потому что она предтренированная и явно не видела новые идеи. Задача [подсчет анаграмм](https://leetcode.com/problems/count-anagrams/) (добавлена в декабре 2022 года)

In [148]:
leetcode = '''\
/**
<insert>
*/
class Solution {
    public int countAnagrams(String s) {
        final int mod = 1_000_000_007; 
        int n = s.length(); 
        long[] fact = new long[n+1], ifact = new long[n+1], inv = new long[n+1]; 
        fact[0] = ifact[0] = inv[0] = inv[1] = 1; 
        for (int x = 1; x <= n; ++x) {
            if (x >= 2) inv[x] = mod - mod/x * inv[mod%x] % mod; 
            fact[x] = fact[x-1] * x % mod; 
            ifact[x] = ifact[x-1] * inv[x] % mod; 
        }
        long ans = 1; 
        for (var buff : s.split(" ")) {
            ans = ans * fact[buff.length()] % mod; 
            int[] freq = new int[26]; 
            for (var ch : buff.toCharArray())  ++freq[ch-'a']; 
            for (var x : freq) ans = ans * ifact[x] % mod; 
        }
        return (int) ans; 
    }
}'''

In [150]:
docstring = code_to_docstring(leetcode)
print(docstring)

/**

*/
class Solution {
    public int countAnagrams(String s) {
        final int mod = 1_000_000_007; 
        int n = s.length(); 
        long[] fact = new long[n+1], ifact = new long[n+1], inv = new long[n+1]; 
        fact[0] = ifact[0] = inv[0] = inv[1] = 1; 
        for (int x = 1; x <= n; ++x) {
            if (x >= 2) inv[x] = mod - mod/x * inv[mod%x] % mod; 
            fact[x] = fact[x-1] * x % mod; 
            ifact[x] = ifact[x-1] * inv[x] % mod; 
        }
        long ans = 1; 
        for (var buff : s.split(" ")) {
            ans = ans * fact[buff.length()] % mod; 
            int[] freq = new int[26]; 
            for (var ch : buff.toCharArray())  ++freq[ch-'a']; 
            for (var x : freq) ans = ans * ifact[x] % mod; 
        }
        return (int) ans; 
    }
}


Ну как бы да. Давайте еще раз

In [151]:
docstring = code_to_docstring(leetcode)
print(docstring)

/**

*/
class Solution {
    public int countAnagrams(String s) {
        final int mod = 1_000_000_007; 
        int n = s.length(); 
        long[] fact = new long[n+1], ifact = new long[n+1], inv = new long[n+1]; 
        fact[0] = ifact[0] = inv[0] = inv[1] = 1; 
        for (int x = 1; x <= n; ++x) {
            if (x >= 2) inv[x] = mod - mod/x * inv[mod%x] % mod; 
            fact[x] = fact[x-1] * x % mod; 
            ifact[x] = ifact[x-1] * inv[x] % mod; 
        }
        long ans = 1; 
        for (var buff : s.split(" ")) {
            ans = ans * fact[buff.length()] % mod; 
            int[] freq = new int[26]; 
            for (var ch : buff.toCharArray())  ++freq[ch-'a']; 
            for (var x : freq) ans = ans * ifact[x] % mod; 
        }
        return (int) ans; 
    }
}


Снова ничего, хотя даже в названии есть подсказка. Теперь давайте наверняка

In [156]:
%%time
N = 10
anagram_docstrings = []

progress = IntProgress(min=0, max=N)
display(progress)

for i in range(N):
    docstring = code_to_docstring(leetcode)
    anagram_docstrings.append(docstring)
    progress.value += 1

IntProgress(value=0, max=10)

CPU times: user 9min 19s, sys: 2.85 s, total: 9min 21s
Wall time: 4min 40s


In [164]:
import re

success = dict()

for i, ds in enumerate(anagram_docstrings):
    mo = re.search(".+anagram.+", ds)
    if mo:
        success[i] = mo.group()

In [165]:
success

{}

Вообщем, вердикт такой, что нет смысла использовать ```InCoder``` для суммаризации кода в серьезных задачах, потому что возможности небольшие, - то есть получается решать задачу там где код вполне читаем. Это обусловленно тем что модель неиндуктивна. Человеку в таких ситуациях понадобится несколько секунд чтобы понять что происходит. Для использования в учебных целях окей.